# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), October 4, 2018**

In [1]:
using DataFrames # load package

## Reshaping DataFrames

### Wide to long

In [2]:
x = DataFrame(id=[1,2,3,4], id2=[1,1,2,2], M1=[11,12,13,14], M2=[111,112,113,114])

,id,id2,M1,M2
,Int64,Int64,Int64,Int64
1,1,1,11,111
2,2,1,12,112
3,3,2,13,113
4,4,2,14,114


In [3]:
melt(x, :id, [:M1, :M2]) # first pass id-variables and then measure variables; meltdf makes a view

,variable,value,id
,Symbol,Int64,Int64
1,M1,11,1
2,M1,12,2
3,M1,13,3
4,M1,14,4
5,M2,111,1
6,M2,112,2
7,M2,113,3
8,M2,114,4


In [4]:
# optionally you can rename columns; melt and stack are identical but order of arguments is reversed
stack(x, [:M1, :M2], :id, variable_name=:key, value_name=:observed) # first measures and then id-s; stackdf creates view

,key,observed,id
,Symbol,Int64,Int64
1,M1,11,1
2,M1,12,2
3,M1,13,3
4,M1,14,4
5,M2,111,1
6,M2,112,2
7,M2,113,3
8,M2,114,4


In [5]:
# if second argument is omitted in melt or stack , all other columns are assumed to be the second argument
# but measure variables are selected only if they are <: AbstractFloat
melt(x, [:id, :id2])

,variable,value,id,id2
,Symbol,Int64,Int64,Int64
1,M1,11,1,1
2,M1,12,2,1
3,M1,13,3,2
4,M1,14,4,2
5,M2,111,1,1
6,M2,112,2,1
7,M2,113,3,2
8,M2,114,4,2


In [6]:
melt(x, [1, 2]) # you can use index instead of symbol

,variable,value,id,id2
,Symbol,Int64,Int64,Int64
1,M1,11,1,1
2,M1,12,2,1
3,M1,13,3,2
4,M1,14,4,2
5,M2,111,1,1
6,M2,112,2,1
7,M2,113,3,2
8,M2,114,4,2


In [7]:
x = DataFrame(id = [1,1,1], id2=['a','b','c'], a1 = rand(3), a2 = rand(3))

,id,id2,a1,a2
,Int64,Char,Float64,Float64
1,1,'a',0.883811,0.347507
2,1,'b',0.221289,0.948051
3,1,'c',0.743618,0.298657


In [8]:
melt(x)

,variable,value,id,id2
,Symbol,Float64,Int64,Char
1,a1,0.883811,1,'a'
2,a1,0.221289,1,'b'
3,a1,0.743618,1,'c'
4,a2,0.347507,1,'a'
5,a2,0.948051,1,'b'
6,a2,0.298657,1,'c'


In [9]:
melt(DataFrame(rand(3,2))) # by default stack and melt treats floats as value columns

,variable,value
,Symbol,Float64
1,x1,0.320536
2,x1,0.94699
3,x1,0.999543
4,x2,0.484661
5,x2,0.792754
6,x2,0.214669


In [10]:
df = DataFrame(rand(3,2))
df[:key] = [1,1,1]
mdf = melt(df) # duplicates in key are silently accepted

,variable,value,key
,Symbol,Float64,Int64
1,x1,0.81656,1
2,x1,0.610571,1
3,x1,0.532249,1
4,x2,0.574836,1
5,x2,0.90107,1
6,x2,0.682834,1


### Long to wide

In [11]:
x = DataFrame(id = [1,1,1], id2=['a','b','c'], a1 = rand(3), a2 = rand(3))

,id,id2,a1,a2
,Int64,Char,Float64,Float64
1,1,'a',0.61586,0.998759
2,1,'b',0.432679,0.686297
3,1,'c',0.226318,0.811977


In [12]:
y = melt(x, [1,2])
display(x)
display(y)

,id,id2,a1,a2
,Int64,Char,Float64,Float64
1,1,'a',0.61586,0.998759
2,1,'b',0.432679,0.686297
3,1,'c',0.226318,0.811977


,variable,value,id,id2
,Symbol,Float64,Int64,Char
1,a1,0.61586,1,'a'
2,a1,0.432679,1,'b'
3,a1,0.226318,1,'c'
4,a2,0.998759,1,'a'
5,a2,0.686297,1,'b'
6,a2,0.811977,1,'c'


In [13]:
unstack(y, :id2, :variable, :value) # stndard unstack with a unique key

,id2,a1,a2
,Char,Float64⍰,Float64⍰
1,'a',0.61586,0.998759
2,'b',0.432679,0.686297
3,'c',0.226318,0.811977


In [14]:
unstack(y, :variable, :value) # all other columns are treated as keys

,id,id2,a1,a2
,Int64,Char,Float64⍰,Float64⍰
1,1,'a',0.61586,0.998759
2,1,'b',0.432679,0.686297
3,1,'c',0.226318,0.811977


In [15]:
# by default :id, :variable and :value names are assumed; in this case it produces duplicate keys and last values are taken
# this is deprecated - in the future all columns other than :variable and :value will be treated as keys
unstack(y)

┌ Warning: In the future `unstack(df)` will call `unstack(df, :variable, :value)`. use `unstack(df, :id, :variable, :value)` to treat `:id` as the only `rowkeys` column
│   caller = top-level scope at In[15]:1
└ @ Core In[15]:1
┌ Warning: Duplicate entries in unstack at row 2 for key 1 and variable a1.
└ @ DataFrames C:\Users\bogumil.kaminski\.julia\packages\DataFrames\1PqZ3\src\abstractdataframe\reshape.jl:244


,id,a1,a2
,Int64,Float64⍰,Float64⍰
1,1,0.226318,0.811977


In [16]:
df = stack(DataFrame(rand(3,2)))

,variable,value
,Symbol,Float64
1,x1,0.0534741
2,x1,0.196309
3,x1,0.912957
4,x2,0.505152
5,x2,0.614941
6,x2,0.191787


In [17]:
unstack(df, :variable, :value) # unable to unstack when no key column is present

ArgumentError: ArgumentError: No key column found